# 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!

最後一天的作業請閱讀這篇非常詳盡的文章CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以參考

In [9]:
from keras import backend as K
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential

In [10]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
x_test shape: (10000, 32, 32, 3)
50000 train samples
10000 test samples


In [11]:
IMAGE_SIZE = (32, 32)


rest_model = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(32,32,3))

    
x = rest_model.output
x = Flatten()(x)

# 增加 DropOut layer
#last_fe = Dropout(0.5)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(10, activation='softmax', name='softmax')(x)

newModel = Model(inputs=rest_model.input,outputs=output_layer)

newModel.summary()

//anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
____________________________________________________________________________________________

In [12]:
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 20 # 訓練整個資料集共 20個循環

import keras.losses as kl

for layer in newModel.layers[:2]:
    layer.trainable = False
for layer in newModel.layers[2:]:
    layer.trainable = True

newModel.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-5),
              metrics=['accuracy'])

In [13]:
history = newModel.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = newModel.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 1980s 40ms/step - loss: 2.4401 - accuracy: 0.2636 - val_loss: 5.8251 - val_accuracy: 0.1445
Epoch 2/20
50000/50000 [==============================] - 1918s 38ms/step - loss: 1.5246 - accuracy: 0.4855 - val_loss: 1.4747 - val_accuracy: 0.5244
Epoch 3/20
50000/50000 [==============================] - 1928s 39ms/step - loss: 1.1939 - accuracy: 0.5933 - val_loss: 1.2715 - val_accuracy: 0.5788
Epoch 4/20
50000/50000 [==============================] - 1928s 39ms/step - loss: 0.9926 - accuracy: 0.6562 - val_loss: 1.1608 - val_accuracy: 0.6160
Epoch 5/20
50000/50000 [==============================] - 1926s 39ms/step - loss: 0.8445 - accuracy: 0.7076 - val_loss: 1.0748 - val_accuracy: 0.6445
Epoch 6/20
50000/50000 [==============================] - 1925s 39ms/step - loss: 0.7133 - accuracy: 0.7543 - val_loss: 1.0381 - val_accuracy: 0.6603
Epoch 7/20
50000/50000 [==========================